<a href="https://colab.research.google.com/github/JaskiratSingh10/CS503/blob/main/EvalAI2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux"

In [ ]:
# !pip install --upgrade torch torchvision torchaudio
# !pip install --upgrade transformers

LIBRARY IMPORTS

In [ ]:
import PyPDF2
import re

In [ ]:
#for summarization
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('punkt_tab', quiet=True)


True

Extracting PDF Text

In [ ]:
def extract_pdf_text(pdf_path):
    full_text = ""
    try:
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text = page.extract_text()
                if text:
                    full_text += text + "\n"
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")

    return full_text


Upload PDF

In [ ]:
pdf_file = "/content/R008.pdf"  # Replace with your PDF file path
text = extract_pdf_text(pdf_file)
print("Extracted Text:")
print(text)

Extracted Text:
Advanced techniques for through and contextually
Interpreting Noun-Noun Compounds
Abstract
This study examines the effectiveness of transfer learning and multi-task learning
in the context of a complex semantic classification problem: understanding the
meaning of noun-noun compounds. Through a series of detailed experiments and
an in-depth analysis of errors, we demonstrate that employing transfer learning by
initializing parameters and multi-task learning through parameter sharing enables a
neural classification model to better generalize across a dataset characterized by a
highly uneven distribution of semantic relationships. Furthermore, we illustrate
how utilizing dual annotations, which involve two distinct sets of relations applied
to the same compounds, can enhance the overall precision of a neural classifier and
improve its F1 scores for less common yet more challenging semantic relations.
1 Introduction
Noun-noun compound interpretation involves determining the

In [ ]:
def check_pdf_pages(pdf_path):
  try:
        reader=PyPDF2.PdfReader(pdf_file)
        page_count = len(reader.pages)
        if 4 <= page_count <= 12:
          print(f"File: {reader} -> Fine (Number of pages: {page_count})")
          return True
        else:
          if page_count < 4:
            print(f"File: {reader} -> Reject (Number of pages: {page_count}) RESEARCH PAPER IS TOO SHORT")
            return False
          else:
            print(f"File: {reader} -> Reject (Number of pages: {page_count}) EXCEEDS THE RECOMMENDED COUNT")
          return False
  except Exception as e:
        return f"Error: {e}"

In [ ]:
check_pdf_pages(pdf_file)

File: <PyPDF2._reader.PdfReader object at 0x78e3f8173a10> -> Fine (Number of pages: 9)


True

In [ ]:
output_text_path = "output.txt"
# Save the extracted text to a file
with open(output_text_path, "w", encoding="utf-8") as text_file:
    text_file.write(text)

In [ ]:
import re

# Define synonyms to map extracted headings to our desired keys.
heading_synonyms = {
    'abstract': 'abstract',
    'related work': 'related work',
    'introduction': 'introduction',
    'methodology': 'methods',
    'methods': 'methods',
    'dataset': 'methods',
    'results': 'experiments',
    'experiments': 'experiments',
    'conclusions': 'conclusion',
    'conclusion': 'conclusion',
    'appendix': 'appendix',
    'appendices': 'appendix'
}

# Our target headings including the new "abstract" heading.
target_headings = ["abstract", "related work","introduction", "methods", "experiments", "conclusion"]

# Initialize the dictionary with empty strings for each key.
sections = {key: "" for key in target_headings}

# Open and read the text file.
with open("output.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

current_section = None

# Regex pattern for a main heading line starting with a number (e.g., "1 Introduction" or "2 Methodology").
numbered_heading_pattern = re.compile(r'^(\d+)\s+(.+)$')
# Regex pattern for detecting a standalone "abstract" heading (case-insensitive).
abstract_pattern = re.compile(r'^abstract$', re.IGNORECASE)

for line in lines:
    line = line.strip()
    if not line:
        continue

    # Check for a standalone "abstract" heading.
    if abstract_pattern.match(line):
        current_section = "abstract"
        continue

    # Check if the line looks like a numbered main heading.
    match = numbered_heading_pattern.match(line)
    if match:
        _, title = match.groups()
        title_norm = title.lower().strip()
        mapped_key = None
        # Check if any of our synonyms appear in the heading.
        for syn, key in heading_synonyms.items():
            if syn in title_norm:
                mapped_key = key
                break
        # If the mapped key is one of our target sections, switch to that section.
        if mapped_key in target_headings:
            current_section = mapped_key
            continue  # Skip adding the heading line itself.

    # If a section has been set, add the current line to its content.
    if current_section:
        sections[current_section] += line + " "

# Optionally, strip extra whitespace from each section.
sections = {k: v.strip() for k, v in sections.items()}

# Print the resulting mapping.
for key, text in sections.items():
    print(f"--- {key.upper()} ---")
    print(text)
    print()


--- ABSTRACT ---
This study examines the effectiveness of transfer learning and multi-task learning in the context of a complex semantic classification problem: understanding the meaning of noun-noun compounds. Through a series of detailed experiments and an in-depth analysis of errors, we demonstrate that employing transfer learning by initializing parameters and multi-task learning through parameter sharing enables a neural classification model to better generalize across a dataset characterized by a highly uneven distribution of semantic relationships. Furthermore, we illustrate how utilizing dual annotations, which involve two distinct sets of relations applied to the same compounds, can enhance the overall precision of a neural classifier and improve its F1 scores for less common yet more challenging semantic relations.

--- RELATED WORK ---
Approaches to interpreting noun-noun compounds differ based on the classification of compound relations, as well as the machine learning mode

Summarizing bigger sections


In [ ]:
def summarize_text(text, target_words=200):
    """
    Summarizes the given text to around the specified target word count,
    preserving technical information by prioritizing sentences with important terms.
    """
    sentences = sent_tokenize(text)
    if not sentences:
        return text

    # Tokenize words and remove stopwords to identify significant terms
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]

    # Calculate word frequency to determine importance
    word_freq = nltk.FreqDist(filtered_words)

    # Score sentences based on the sum of word frequencies
    sentence_scores = {}
    for idx, sentence in enumerate(sentences):
        for word in word_tokenize(sentence.lower()):
            if word in word_freq:
                sentence_scores[idx] = sentence_scores.get(idx, 0) + word_freq[word]

    # Sort sentences by their scores in descending order
    sorted_sentences = sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True)

    selected_indices = []
    current_word_count = 0
    for idx, score in sorted_sentences:
        sentence = sentences[idx]
        sentence_words = word_tokenize(sentence)
        sentence_word_count = len(sentence_words)

        if current_word_count + sentence_word_count <= target_words:
            selected_indices.append(idx)
            current_word_count += sentence_word_count
        else:
            remaining = target_words - current_word_count
            if remaining > 0:
                # Add partial sentence if it contributes significantly (optional)
                selected_indices.append(idx)
                current_word_count += sentence_word_count  # May exceed slightly
            break

    # Sort selected indices to maintain original order for coherence
    selected_indices = sorted(selected_indices)
    summary = ' '.join([sentences[i] for i in selected_indices])

    # Ensure the summary is close to the target word count
    summary_words = word_tokenize(summary)
    if len(summary_words) > target_words:
        summary = ' '.join(summary_words[:target_words])

    return summary

In [ ]:
def process_sections(sections_dict):
    """
    function iterates through each entry in the dictionary, using word_tokenize to accurately count words
    """
    for key in sections_dict:
        content = sections_dict[key]
        words = word_tokenize(content)
        if len(words) > 350:
            summarized_content = summarize_text(content)
            sections_dict[key] = summarized_content
    return sections_dict

In [ ]:
sections=process_sections(sections)
print(sections['abstract'])
print('-----------')
print(sections['methods'])

This study examines the effectiveness of transfer learning and multi-task learning in the context of a complex semantic classification problem: understanding the meaning of noun-noun compounds. Through a series of detailed experiments and an in-depth analysis of errors, we demonstrate that employing transfer learning by initializing parameters and multi-task learning through parameter sharing enables a neural classification model to better generalize across a dataset characterized by a highly uneven distribution of semantic relationships. Furthermore, we illustrate how utilizing dual annotations, which involve two distinct sets of relations applied to the same compounds, can enhance the overall precision of a neural classifier and improve its F1 scores for less common yet more challenging semantic relations.
-----------
We chose this dataset for two primary reasons : firstly , the dual annotation of relations on the same set of compounds is ideal for exploring TL and MTL approaches ; s

**LOADING LLM**

In [ ]:
!huggingface-cli login #


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `data titans` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `data titans`


In [ ]:
import torch
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))


CUDA Available: True
GPU Name: Tesla T4


In [ ]:
# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
model_id = "google/gemma-3-4b-it"
model = AutoModelForCausalLM.from_pretrained(
    model_id,device_map="auto",
    torch_dtype="bfloat16",
    attn_implementation="sdpa"

)
# Load tokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)
chat_pipeline = pipeline(
    "text-generation",
    model=model,
    temperature=0.5,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.eos_token_id,
    model_kwargs={"use_cache": True}  # Critical for speed
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:1682: FutureWarning: Loading a multimodal model with `AutoModelForCausalLM` is deprecated and will be removed in v5. `AutoModelForCausalLM` will be used to load only the text-to-text generation module.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Device set to use cuda:0


**QnA**

In [ ]:
checklist_questions = {
    "1a":"Whether the problem statement is clear, well-articulated, and addresses important issues in the field?",
    "1b": "Do the authors describe the limitations of their work?",
    "1c": "Do the authors discuss any potential societal impacts of their work?",
    "1d": "Prior literature has been analysed? Is the work well motivated and appropriately grounded in theory?",
    "2a": "If the authors include theoretical results, do the authors state the full set of assumptions of all theoretical results?",
    "2b": "If the authors include theoretical results, do the authors include complete proofs of all theoretical results?",
    "3a": "If the authors ran experiments, do the authors include the code, data, and instructions needed to reproduce the main experimental results (either in the supplemental material or as a URL)?",
    "3b": "If the authors ran experiments, do the authors specify all the training details (e.g., data splits, hyperparameters, how they were chosen)?",
    "3d": "If the authors ran experiments, do the authors include the total amount of compute and the type of resources used (e.g., type of GPUs, internal cluster, or cloud provider)?",
    "4a": "If the authors use existing assets (e.g., code, data, models), do the authors cite the creators or mentioned the license of assests?",
    "4c": "If the authors curate/release new assets (e.g., code, data, models), do the authors include any new assets either in the supplemental material or as a URL?",
    "4d": "If the authors curate/release new assets (e.g., code, data, models), do the authors discuss whether and how consent was obtained from people whose data they are using/curating?",
    "5a": "If the authors used crowdsourcing or conducted research with human subjects, do the authors include the full text of instructions given to participants and screenshots, if applicable?"
}

# Map each checklist question to the sections to be used as context
question_section_mapping = {
    "1a":["abstract"],
    "1b": ["conclusion"],
    "1c": ["conclusion","introduction"],
    "1d": ["related work","introduction"],
    "2a": ["methods"],
    "2b": ["methods"],
    "3a": ["experiments"],
    "3b": ["experiments"],
    "3d": ["experiments"],
    "4a": ["methods"],
    "4b": ["methods"],
    "4c": ["methods"],
    "4d": ["methods"],
    "5a": ["methods"]
}


In [ ]:
def generate_answers(prompts, max_new_tokens=250, temperature=0.5):
    """
    Generates answers from the LLM for a list of prompts using batching.

    Parameters:
        prompts (list): List of prompt strings.
        max_new_tokens (int): Maximum tokens to generate for each prompt.
        temperature (float): Sampling temperature.

    Returns:
        list: A list of generated answers, one per prompt.
    """
    # Batch process the list of prompts
    results = chat_pipeline(prompts, max_new_tokens=max_new_tokens, temperature=temperature,top_p=1.0)
    answers = []
    for prompt, result in zip(prompts, results):
        generated_text = result[0]['generated_text']
        # Remove the prompt from the generated text, if it is repeated in the output
        answer = generated_text[len(prompt):].strip()
        answers.append(answer)
    return answers

In [ ]:
def answer_all_checklist_questions(sections_text):
    """
    For each checklist question, combine the context from the corresponding sections,
    generate an answer from the LLM (in batch), and return a dictionary mapping the checklist
    question (key) to its answer (value).

    Parameters:
        sections_text (dict): A dictionary mapping section names to their extracted text.
                              e.g., {"introduction": "...", "methods": "...", "conclusion": "..."}

    Returns:
        dict: A dictionary mapping each checklist question to its generated answer.
    """
    prompts = []
    questions_list = []  # to keep track of the question text for mapping later

    for key, question in checklist_questions.items():
        # Retrieve the sections that correspond to this question
        section_keys = question_section_mapping.get(key, [])
        context_parts = []
        for sec in section_keys:
            if sec in sections_text:
                context_parts.append(f"--- {sec.upper()} ---\n{sections_text[sec]}")
            else:
                print(f"Warning: No text provided for section '{sec}'")
        context = "\n\n".join(context_parts)

        # Construct the prompt for the LLM
        prompt = (
            "Below is context extracted from a research paper:\n\n"
            f"{context}\n\n"
            "Answer the following question with 'yes', 'no', or 'n/a' and provide a brief justification for your answer.\n"
            f"Question: {question}\n\n"
            "Answer:"
        )
        prompts.append(prompt)
        questions_list.append(question)

    # Generate answers in batch using the GPU
    answers_batch = generate_answers(prompts)

    # Map each question to its answer
    answers = {}
    for question, answer in zip(questions_list, answers_batch):
        print(question, answer)
        print('-------********-----------')
        answers[question] = answer


    return answers

In [ ]:
output_dict=answer_all_checklist_questions(sections)
print(output_dict)

Whether the problem statement is clear, well-articulated, and addresses important issues in the field? yes
Justification: The abstract clearly states the problem being addressed (understanding the meaning of noun-noun compounds), the methods used (transfer learning and multi-task learning), the benefits observed (improved generalization, enhanced precision, and F1 scores), and the challenges encountered (uneven distribution of semantic relationships). It presents a focused and relevant research question.
-------********-----------
Do the authors describe the limitations of their work? yes
Justification: The authors explicitly state, "However, clear indications of ’lexical memorization’ effects are evident in our error analysis of unseen compounds," and "Furthermore, in multi-task learning, the complete sharing of model architecture across tasks degrades its capacity to generalize when it comes to less frequent relations." These are clear descriptions of limitations.
-------********----

Again calling LLM to generate the final review based on the QnA

In [ ]:
def generate_review_from_qna(output_dict, chat_pipeline, temperature=0.3):
    """
    Generates a paper review based on a dictionary of checklist Q&A.

    Parameters:
        output_dict (dict): A dictionary where keys are checklist questions (or question keys)
                            and values are the corresponding answers generated by the LLM.
        chat_pipeline: The Hugging Face text-generation pipeline (or similar callable) for generating output
        temperature (float): Sampling temperature for generation.

    Returns:
        str: The generated review of the paper.
    """
    # Constructing a prompt by combining each Q&A pair.
    prompt = "Below is the checklist Q&A for a research paper:\n"
    for q, a in output_dict.items():
        prompt += f"\nQ: {q}\nA: {a}\n"

    prompt += ("\n"
               "You are a highly knowledgeable and meticulous reviewer for prestigious AI conferences. generate a comprehensive review of the paper."
               "Your review should  summarize in one pargraph some strengths and weaknesses of the paper and appreciate the strengths"
               "and also Comment on experimental design and data presentation and Suggestions for improvements ")

    # Generate the review using the provided text-generation pipeline.
    result = chat_pipeline(prompt,max_new_tokens=700,do_sample=False, temperature=temperature)
    generated_text = result[0]['generated_text']

    # remove the prompt from the generated text as it is repeated.
    review = generated_text[len(prompt):].strip()
    return review

In [ ]:
generated_review = generate_review_from_qna(output_dict, chat_pipeline)
print(generated_review)

Here's my review of the paper:

This research paper presents a valuable exploration of applying transfer learning (TL) and multi-task learning (MTL) to the challenging task of noun-noun compound interpretation in Natural Language Processing. The paper’s strength lies in its clear articulation of the problem, a well-motivated approach based on established theories of TL and MTL, and a thorough examination of different model configurations. The authors effectively highlight the limitations of existing approaches and demonstrate the potential benefits of their proposed methods, particularly in terms of improved generalization and F1 scores. However, the paper suffers from a lack of detail regarding the experimental setup, including specific training hyperparameters, data splits, and computational resources utilized. The absence of code, data, and instructions hinders reproducibility and limits the broader impact of the research. Furthermore, the paper’s discussion of societal impacts is e